## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd
import numpy as np

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🪄 Fetch Feature Groups </span>

In [ ]:
users_fg = fs.get_feature_group(
    name="users",
    version=1,
)

videos_fg = fs.get_feature_group(
    name="videos",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

interactions_month_sincos_fg = fs.get_feature_group(
    name="interactions_month_sincos",
    version=1,
)

query = interactions_fg.select_except(['interaction_id',  'watch_time', 'interaction_date', 'category_id'])\
               .join(interactions_month_sincos_fg.select(['month_cos', 'month_sin']))\
               .join(users_fg.select_except(['upload_date', 'upload_month'])) \
               .join(videos_fg.select_except(['registration_date', 'registration_month']))


In [ ]:
ranking_df = query.read()

In [ ]:
ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']), 
    1, 
    0,
)

ranking_df = ranking_df[["user_id", "video_id", "category_id", "interaction_month", "video_length", "gender", "age", "country", "label"]]

## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>

In [ ]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    version=1,
    description="Ranking Data.",
    primary_key=["user_id", "video_id"],
    partition_key = ["interaction_month"],    
    online_enabled=False, 
    parents=[users_fg, videos_fg, interactions_fg, interactions_month_sincos_fg],
    statistics_config = {
    "enabled": True,
    "histograms": True,
    "correlations": True,
    }
)

ranking_fg.insert(ranking_df)
print('Done ✅')

In [ ]:
feature_descriptions = [
    {"name": "user_id", "description": "Unique identifier for each user."},
    {"name": "video_id", "description": "Identifier for the video."},
    {"name": "category_id", "description": "Id of the video category."},   
    {"name": "interaction_month", "description": "Month of interaction, derived from interaction_date."}, 
    {"name": "video_length", "description": "Video length in sconds."},
    {"name": "gender", "description": "Gender of the user."},
    {"name": "age", "description": "Age of the user."},
    {"name": "country", "description": "Country of Residence of the user."},
    {"name": "label", "description": "Label indicating whether the article was purchased (1) or not (0)."},
]

for desc in feature_descriptions: 
    ranking_fg.update_feature_description(desc["name"], desc["description"])

---